# Obsługa ROS Topics

## Wprowadzenie - polecenia w terminalu
ROS Topics używane są do komunikacji rozgłoszeniowej. Nie ma znaczenia kto jest nadawcą, a kto odbiorcą wiadomości. Za kodowanie przesyłanej wiadomości odpowiada **Publisher**, a za rozkodowanie **Subscriber**. Typy wiadomości przechowywane są w katalogu srv, a rozszerzenie wiadomości to .msg.

### Struktura wiadomości

Z lewej strony należy podać typ wiadomości ROS. Mogą być one bardziej złożone i składać się z już utworzonych wiadomości (ROS msg). Z prawej strony podawana jest nazwa pola.

Typ wiadomości to nazwa paczki + nazwa_wiadomosci.msg. Wyświetlenie przykładowej wiadomości znajdującej się w paczce tsr_materials:

In [ ]:
!rosmsg info pkg_tsr/RobotInfo

### Wyświetlenie listy aktualnie dostępnych topiców

In [ ]:
!rostopic list

### Wyświetlenie dostępnej pomocy dla polecenia rostopic

In [ ]:
!rostopic --help

### Generowane topic'i przez node'a turtlesim_node

Dla pojedynczego utworzonego robota w przestrzeni nazw na przykładzie turtle1 dostępne są nastpujące topic'i:
- /turtle1/cmd_vel - prędkości sterujące robotem
- /turtle1/color_sensor - kolor
- /turtle1/pose - położenie robota

Wyświetlenie informacji o topic'u **/turtle1/cmd_vel**:

In [ ]:
!rostopic info /turtle1/cmd_vel

#### Zadanie 1
Wyświetlić informację dla pozostałych topic'ów.

#### Zadanie 2
Wyświetlić informację o typach powyższych wiadomości.

### Sprawdzenie danych w wiadomości

In [ ]:
# po wywołaniu szybk zatrzymać stopem
# podgląd całej wiadomości
!rostopic echo -c /turtle1/pose

In [ ]:
# podgląd pojedynczego pola
!rostopic echo -c /turtle1/pose/theta

## Publisher - Python

Podstawową biblioteką do obsługi ROS w Pythonie jest **rospy**. Importowanie wiadomości na podstawie informacji o typie wiadomości jest następujące:

**import** ***nazwa_paczki.msg*** **import** ***typ_wiadomosci***

In [ ]:
import rospy
from geometry_msgs.msg import Twist

Inicjalizacja node'a, aby ROS mógł jednoznacznie rozpoznać node'a.

Uwaga techniczna. 1 init_node wywoływany w danym zeszycie od Jupyter Notebook.

In [ ]:
rospy.init_node("topics_test", anonymous=True)

Do utworzenia publishera wykorzystywana jest klasa *Publisher* z biblioteki *rospy*. Przyjmowane kolejno argumenty:
- nazwa topic'a, 
- typ wiadomości, 
- liczba zakolejkowanych wiadomości.

In [ ]:
pub_speed=rospy.Publisher("/turtle1/cmd_vel",Twist,queue_size=10)

Utworzenie i uzupełnienie wiadomości.

In [ ]:
msg = Twist()
msg.linear.x = 0.6
msg.angular.z = 1

Do wysłania wiadomości do robota *turtle1* jest metoda klasy *Publisher* o nazwie *publish*, która jako argument
przyjmuje typ oczekiwanej wiadomości.

In [ ]:
pub_speed.publish(msg)

#### Zadanie 3
Wysyłając różne prędkości sterujące narysować jakiś ciekawy kształ robotem. Na początek może to być kwadrat, trójkąt lub jakaś bardziej skomplikowana abstrakcja. Szybkie wysłanie kolejnych wiadomości po sobie spowoduje, że
wartości prędkości zostaną bardzo szybko wysłane zanim robot zdąży wykonać ruch. W tym celu można użyć funkcji:

**rospy.sleep(czas_w_sekundach)**

Jej zadaniem jest oczekiwanie określonego czasu przed wykonaniem kolejnej akcji

#### Zadanie 4
Opublikować dowolną wiadomość tekstową na topicu "informacja". Wykorzystać wiadomość typu **String** z biblioteki **std_msgs**.
1. wyświetlić wiadomość typu string
2. zaimportować wiadomość typu string
3. utworzyć zmienną publishera
4. utworzyć wiadomość i uzupełnić dowolnym tekstem
5. wysłać wiadomość
6. Podejrzeć w terminalu czy na topicu pojawia się wysłany tekst

## Subscriber - Python

In [ ]:
!rosservice call reset

In [ ]:
def callback_function(msg_data):
    print("Subscriber - otrzymana wiadomosc: ", msg_data.data)

Do utworzenia subscribera wykorzystywana jest klasa Publisher z biblioteki rospy. Przyjmowane kolejno argumenty:
- nazwa topic'u
- typ wiadomości
- nazwa funkcji, która jest wywoływana do odebrania danych z odczytanej wiadomości

In [ ]:
my_subscriber = rospy.Subscriber("informacja",String,callback_function)

Wyłączenie subscriber'a.

In [ ]:
my_subscriber.unregister()

#### Zadanie 5
Utworzyć subscriber'a odpowiedzialnego za odczyt położenia żółwia.
1. Sprawdzić typ wiadomości
2. Zaimportować wiadomość od położenia
3. napisać funkcję odczytującą wartość położenia i zapisującą ją w zmiennej globalnej. Nie wyświetlać w funkcji, gdyż przychodzi bardzo dużo wiadomości.
4. utworzyć zmienną subscriber'a 
5. Wyświetlić wartość zmiennej globalnej

## Jednoczesny Publisher Subscriber - przykład

In [ ]:
# jeśli to konieczne zaimportować odpowiednie biblioteki

vel_topic_name = ... # UZUPEŁNIĆ
vel_topic_type = ... # UZUPEŁNIĆ
pub_velocity = rospy.Publisher(vel_topic_name,vel_topic_type,queue_size=10)

direction_right = True
def robot_control(message):    
    """Analiza wiadomości i wysłanie jej na innym topicu"""
    global direction_right
    vel_msg = Twist()
    if direction_right:
        vel_msg.linear.x = 0.5
        vel_msg.angular.z = 0
    else:
        vel_msg.linear.x = -0.5
        vel_msg.angular.z = 0
        
    if message.x > 7:
        direction_right = False
    elif message.x < 2:
        direction_right = True
    # wysłanie przeanalizowanych danych    
    pub_velocity.publish(vel_msg)
    


# odebrać wiadomość z topicu /
pose_topic_name = ... # UZUPEŁNIĆ
pose_topic_type = ... # UZUPEŁNIĆ
subscriber= rospy.Subscriber(pose_topic_name, pose_topic_type, robot_control)   